In [1]:
import os
import json
import tensorflow as tf
import tensorflow_hub as hub
import zipfile
import tarfile
import argparse
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.optimizers import SGD

In [2]:
DATA_DIR = "/opt/dkube/input/"
MODEL_DIR = "../model/"
EXTRACT_PATH = "/tmp/data/"
ZIP_FILE = DATA_DIR + "/data.zip"
img_shape = (299,299)

In [3]:
if os.path.exists(ZIP_FILE):
    print("Extracting compressed training data...")
    archive = zipfile.ZipFile(ZIP_FILE)
    for file in archive.namelist():
        if file.startswith('data'):
            archive.extract(file, EXTRACT_PATH)
    print("Training data successfuly extracted")
    DATA_DIR = EXTRACT_PATH + "/data"

Extracting compressed training data...
Training data successfuly extracted


In [4]:
datagen = ImageDataGenerator(rescale=1.0/255.0)
train_it = datagen.flow_from_directory(DATA_DIR + '/train/', class_mode='binary', batch_size=64, target_size=img_shape)
test_it = datagen.flow_from_directory(DATA_DIR + '/valid/', class_mode='binary', batch_size=64, target_size=img_shape)

Found 200 images belonging to 2 classes.
Found 200 images belonging to 2 classes.


In [5]:
model = ResNet50(include_top=False, input_shape=(299, 299, 3))
for layer in model.layers:
    layer.trainable = False
flat1 = Flatten()(model.layers[-1].output)
class1 = Dense(128, activation='relu', kernel_initializer='he_uniform')(flat1)
output = Dense(1, activation='sigmoid')(class1)
# define new model
model = Model(inputs=model.inputs, outputs=output)
# compile model
opt = SGD(lr=0.001, momentum=0.9)
model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])

/usr/local/lib/python3.6/dist-packages/keras_applications/resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '
W1007 12:59:04.506372 139964133111616 deprecation.py:506] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W1007 12:59:04.579623 139964133111616 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/nn_impl.py:180: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.

In [6]:
history = model.fit_generator(train_it, steps_per_epoch=len(train_it), epochs=1, verbose=1)

4/4 [==============================] - 21s 5s/step - loss: 1.2673 - acc: 0.5100


In [7]:
export_path = MODEL_DIR
version = 0
if not tf.io.gfile.exists(export_path):
    tf.io.gfile.makedirs(export_path)
model_contents = tf.io.gfile.listdir(export_path)

saved_models = []
for mdir in model_contents:
    if mdir != 'logs' and mdir != 'metrics':
        saved_models.append(int(mdir))
print(saved_models)
if len(saved_models) < 1:
    version = 1
else:
    version = max(saved_models) + 1
model.save(export_path + 'weights.h5')
tf.keras.backend.set_learning_phase(0)  # Ignore dropout at inference
with tf.keras.backend.get_session() as sess:
    tf.saved_model.simple_save(
        sess,
        export_path + str(version),
        inputs={'input': model.input},
        outputs={'output': model.output})
print("Model saved, version = ", version)

[]


W1007 12:59:41.987456 139964133111616 deprecation.py:323] From <ipython-input-7-2430cd235739>:23: simple_save (from tensorflow.python.saved_model.simple_save) is deprecated and will be removed in a future version.
Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.simple_save.
W1007 12:59:41.988351 139964133111616 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/saved_model/signature_def_utils_impl.py:201: build_tensor_info (from tensorflow.python.saved_model.utils_impl) is deprecated and will be removed in a future version.
Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.utils.build_tensor_info or tf.compat.v1.saved_model.build_tensor_info.


Model saved, version =  1


In [21]:
import cv2
import numpy as np
x = cv2.imread('../data/valid/cats/cat.10026.jpg')
x = cv2.resize(x, (298, 298))
x = np.array(x, dtype=np.float64)
x = x.reshape(1,298,298,3)
res = {"inputs": {'input': x.tolist()}}
with open('cat-inp.json', 'w') as outfile:
    json.dump(res, outfile, indent=4)

In [33]:
import base64
with open("../data/valid/cats/cat.10026.jpg", "rb") as image_file:
    encoded_string = base64.b64encode(image_file.read())

In [37]:
inputs = {
            'signatures':{
                'inputs':[[{'data':encoded_string.decode('utf-8')}]]
            },
            'instances': [],
            'token': 'Dumy token'
}

In [38]:
with open('cat-inp-tansformer.json', 'w') as outfile:
    json.dump(inputs, outfile, indent=4)